In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import torch 
import torch.nn as nn
from torch.utils.data import DataLoader

# Libraries for processing sounds
import librosa
from IPython.display import Audio
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
from AnimalSoundDataset import AnimalSoundDataset

data_path = 'data/Animal_Sound_reduced.csv'
dataset_train = AnimalSoundDataset(data_path, split='train', split_ratio=0.8, seed=42)
dataset_val = AnimalSoundDataset(data_path, split='val', split_ratio=0.8, seed=42)

x_train_list = []
y_train_list = []

loader = DataLoader(dataset_train, batch_size=len(dataset_train))
x_train, y_train = next(iter(loader))

loader = DataLoader(dataset_val, batch_size=len(dataset_val))
x_val, y_val = next(iter(loader))

[WARNING] Failed to extract mel from data/sounds/Lion_1.wav: zero-size array to reduction operation maximum which has no identity


In [8]:
input_dim = 1
n_classes = len(dataset_train.classes)
hyperparameters = dict(input_dim=input_dim,
                    output_dim=n_classes,
                    hidden_layers_size=100,
                    activation='relu',
                    kernel_size_conv=[(57,6),(1,3)],
                    kernel_size_pool=[(4,3),(1,3)],
                    stride_conv=[(1,1),(1,1)],
                    stride_pool=[(1,3),(1,3)],
                    filters=[80,80],
                    batch_normalization=False,
                    dropout_rate=0.5,
                    learning_rate=0.002,
                    batch_size=16,
                    max_epoch = 10)

In [5]:
from ClassesML.AudioModel import AudioModel
model = AudioModel(hyperparameters).to(device)

/Users/anampavicic/miniconda3/envs/DLClass/lib/python3.11/site-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1037.)
  return F.conv2d(


In [6]:
# model = AudioModel(hyperparameters).to(device)
# print(x_val[0])
# y_hat = model(x_val)

In [9]:
from ClassesML.AudioTrainer import AudioTrainer
trainer = AudioTrainer(model, dataset_train, dataset_val, hyperparameters, device=device)
trainer.train()

Epoch 1/10:   0%|          | 0/7 [00:00<?, ?it/s]

[WARNING] Failed to extract mel from data/sounds/Lion_1.wav: zero-size array to reduction operation maximum which has no identity


Epoch 1/10: 100%|██████████| 7/7 [00:06<00:00,  1.09it/s]


Epoch 1: Train Loss: 0.1613, Train Acc: 3.9062, Val Loss: 0.3208, Val Acc: 1.4062


Epoch 2/10:  86%|████████▌ | 6/7 [00:06<00:01,  1.02s/it]

...

Epoch 9: Train Loss: 0.1604, Train Acc: 3.5156, Val Loss: 0.3207, Val Acc: 1.4062


Epoch 10/10:   0%|          | 0/7 [00:00<?, ?it/s]

[WARNING] Failed to extract mel from data/sounds/Lion_1.wav: zero-size array to reduction operation maximum which has no identity


Epoch 10/10: 100%|██████████| 7/7 [00:07<00:00,  1.08s/it]


Epoch 10: Train Loss: 0.1606, Train Acc: 3.1250, Val Loss: 0.3207, Val Acc: 1.4062
